In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm

from sklearn.decomposition import NMF
from scipy.sparse.linalg import svds

#set the max columns to none
pd.set_option('display.max_columns', None)

## I will perform matrix factorization to the interaction matrix

In [3]:
# Read interaction matrix pickle file
type_ = 'validation'
fillna_value = "0"
size = 'demo'
interaction_matrix_file_path = f'./files/pickle/interaction_matrix_{type_}_{size}_{fillna_value}.pkl'
interaction_matrix = pd.read_pickle(interaction_matrix_file_path)
print(interaction_matrix.shape)

(1562, 4074)


In [4]:
interaction_matrix.head(2)

article_id_fixed  7662811  7787495  9052240  9606510  9628753  9638857  \
0                    24.0     13.0     48.0      7.0      2.0    267.0   
1                     0.0      0.0      0.0      0.0      0.0      0.0   

article_id_fixed  9658292  9685856  9695098  9724944  9725978  9735795  \
0                   346.0     63.0     33.0    127.0    151.0      2.0   
1                     0.0      0.0      0.0      0.0     26.0      0.0   

article_id_fixed  9735824  9738457  9741033  9744479  9744480  9745750  \
0                     2.0     12.0     22.0      8.0    333.0    264.0   
1                     0.0      0.0      0.0      0.0      0.0     40.0   

article_id_fixed  9746360  9747369  9747411  9747690  9747803  9748792  \
0                   817.0     42.0    221.0     31.0      8.0    263.0   
1                     0.0      9.0      0.0      0.0      0.0      0.0   

article_id_fixed  9748798  9748916  9748948  9748980  9749076  9749080  \
0                  1014.0    259.0    151.0    214.0    500.0    105.0   
1                     0.0      0.0      0.0      0.0      0.0      0.0   

article_id_fixed  9749156  9749182  9749184  9749224  9749240  9749397  \
0                     5.0    100.0     52.0     56.0     81.0      0.0   
1                     0.0      0.0      0.0      0.0      0.0      0.0   

article_id_fixed  9749444  9749634  9749635  9749643  9749668  9749759  \
0                   996.0      0.0    498.0     62.0     56.0    234.0   
1                     0.0      0.0      0.0      0.0      0.0      0.0   

article_id_fixed  9749886  9749916  9749938  9749947  9749948  9749966  \
0                  2205.0    442.0    180.0    504.0    197.0    229.0   
1                     0.0      0.0      0.0      0.0      0.0      0.0   

article_id_fixed  9750296  9750318  9750687  9750789  9750904  9750995  \
0                   103.0      8.0    656.0    439.0    152.0     33.0   
1                     0.0      0.0      0.0      0.0      0.0      0.0   

article_id_fixed  9751020  9751135  9751220  9751290  9751385  9751411  \
0                   396.0    106.0    140.0    382.0   1060.0    587.0   
1                     0.0      0.0     27.0      0.0      9.0      0.0   

article_id_fixed  9751452  9751524  9751557  9751646  9751670  9751688  \
0                   134.0      0.0    238.0    163.0      0.0    143.0   
1                     0.0      0.0      0.0      0.0     53.0      0.0   

article_id_fixed  9751764  9751771  9751778  9751850  9751866  9751877  \
0                   183.0    348.0     93.0    542.0    184.0     91.0   
1                     0.0      0.0      0.0      0.0      0.0      0.0   

article_id_fixed  9751895  9751897  9751932  9751975  9752155  9752243  \
0                   466.0    861.0    413.0    823.0    927.0    356.0   
1                   127.0      8.0      0.0      0.0      0.0     55.0   

article_id_fixed  9752323  9752340  9752685  9752741  9753047  9753097  \
0                   185.0    188.0    244.0      1.0     68.0      0.0   
1                     0.0      0.0      0.0      0.0      0.0      0.0   

article_id_fixed  9753168  9753177  9753207  9753545  9753603  9753668  \
0                    76.0    374.0     89.0    202.0    108.0    342.0   
1                     0.0      0.0      0.0      8.0      0.0      0.0   

article_id_fixed  9753684  9753811  9753848  9753949  9754042  9754073  \
0                   523.0    380.0     91.0    164.0    103.0    338.0   
1                     0.0      0.0      0.0      0.0      0.0      0.0   

article_id_fixed  9754133  9754158  9754365  9754520  9754603  9754730  \
0                   161.0    231.0    442.0    702.0    161.0    149.0   
1                     0.0      0.0      0.0     41.0      0.0      6.0   

article_id_fixed  9754962  9755027  9755088  9755102  9755285  9755966  \
0                   334.0     89.0    531.0    200.0    160.0    226.0   
1                     0.0      0.0      0.0      0.0      0.0

In [5]:
interaction_matrix.head(2)

article_id_fixed  7662811  7787495  9052240  9606510  9628753  9638857  \
0                    24.0     13.0     48.0      7.0      2.0    267.0   
1                     0.0      0.0      0.0      0.0      0.0      0.0   

article_id_fixed  9658292  9685856  9695098  9724944  9725978  9735795  \
0                   346.0     63.0     33.0    127.0    151.0      2.0   
1                     0.0      0.0      0.0      0.0     26.0      0.0   

article_id_fixed  9735824  9738457  9741033  9744479  9744480  9745750  \
0                     2.0     12.0     22.0      8.0    333.0    264.0   
1                     0.0      0.0      0.0      0.0      0.0     40.0   

article_id_fixed  9746360  9747369  9747411  9747690  9747803  9748792  \
0                   817.0     42.0    221.0     31.0      8.0    263.0   
1                     0.0      9.0      0.0      0.0      0.0      0.0   

article_id_fixed  9748798  9748916  9748948  9748980  9749076  9749080  \
0                  1014.0    259.0    151.0    214.0    500.0    105.0   
1                     0.0      0.0      0.0      0.0      0.0      0.0   

article_id_fixed  9749156  9749182  9749184  9749224  9749240  9749397  \
0                     5.0    100.0     52.0     56.0     81.0      0.0   
1                     0.0      0.0      0.0      0.0      0.0      0.0   

article_id_fixed  9749444  9749634  9749635  9749643  9749668  9749759  \
0                   996.0      0.0    498.0     62.0     56.0    234.0   
1                     0.0      0.0      0.0      0.0      0.0      0.0   

article_id_fixed  9749886  9749916  9749938  9749947  9749948  9749966  \
0                  2205.0    442.0    180.0    504.0    197.0    229.0   
1                     0.0      0.0      0.0      0.0      0.0      0.0   

article_id_fixed  9750296  9750318  9750687  9750789  9750904  9750995  \
0                   103.0      8.0    656.0    439.0    152.0     33.0   
1                     0.0      0.0      0.0      0.0      0.0      0.0   

article_id_fixed  9751020  9751135  9751220  9751290  9751385  9751411  \
0                   396.0    106.0    140.0    382.0   1060.0    587.0   
1                     0.0      0.0     27.0      0.0      9.0      0.0   

article_id_fixed  9751452  9751524  9751557  9751646  9751670  9751688  \
0                   134.0      0.0    238.0    163.0      0.0    143.0   
1                     0.0      0.0      0.0      0.0     53.0      0.0   

article_id_fixed  9751764  9751771  9751778  9751850  9751866  9751877  \
0                   183.0    348.0     93.0    542.0    184.0     91.0   
1                     0.0      0.0      0.0      0.0      0.0      0.0   

article_id_fixed  9751895  9751897  9751932  9751975  9752155  9752243  \
0                   466.0    861.0    413.0    823.0    927.0    356.0   
1                   127.0      8.0      0.0      0.0      0.0     55.0   

article_id_fixed  9752323  9752340  9752685  9752741  9753047  9753097  \
0                   185.0    188.0    244.0      1.0     68.0      0.0   
1                     0.0      0.0      0.0      0.0      0.0      0.0   

article_id_fixed  9753168  9753177  9753207  9753545  9753603  9753668  \
0                    76.0    374.0     89.0    202.0    108.0    342.0   
1                     0.0      0.0      0.0      8.0      0.0      0.0   

article_id_fixed  9753684  9753811  9753848  9753949  9754042  9754073  \
0                   523.0    380.0     91.0    164.0    103.0    338.0   
1                     0.0      0.0      0.0      0.0      0.0      0.0   

article_id_fixed  9754133  9754158  9754365  9754520  9754603  9754730  \
0                   161.0    231.0    442.0    702.0    161.0    149.0   
1                     0.0      0.0      0.0     41.0      0.0      6.0   

article_id_fixed  9754962  9755027  9755088  9755102  9755285  9755966  \
0                   334.0     89.0    531.0    200.0    160.0    226.0   
1                     0.0      0.0      0.0      0.0      0.0

In [6]:
# ---------(NOT USED)-----------------
# # Apply matrix factorization 
# nmf = NMF(n_components=5)
# user_matrix = nmf.fit_transform(interaction_matrix)
# article_matrix = nmf.components_.T


# Perform Singular Value Decomposition (SVD) on the user-article interaction matrix
k = 300  # Number of latent factors (embedding dimensions)

U, sigma, Vt = svds(interaction_matrix.values, k=k)  # Perform SVD, obtaining U, sigma, Vt

# Diagonalize the singular values sigma to create a diagonal matrix
sigma_diag = np.diag(sigma)

# Reconstruct the user and article embeddings
user_matrix = np.dot(U, np.sqrt(sigma_diag))  # User embeddings
article_matrix = np.dot(np.sqrt(sigma_diag), Vt).T  # Article embeddings

In [7]:
print(user_matrix.shape)
print(article_matrix.shape)

(1562, 300)
(4074, 300)


In [8]:
column_lst = interaction_matrix.columns.tolist()

user_matrix_df = pd.DataFrame(user_matrix, index=interaction_matrix.index)
article_matrix_df = pd.DataFrame(article_matrix, index=column_lst)

In [9]:
user_matrix_df.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,0.071445,0.602475,-0.181536,0.989115,1.398682,-0.150730,0.000740,0.491139,-0.354739,0.267759,0.539304,-0.135431,0.482752,0.835472,-0.915800,0.239243,0.597919,-0.336157,-0.088835,0.701762,-0.181855,-0.660545,-1.077204,-0.123642,-0.333158,-0.738075,0.468986,-0.289292,-2.221309,0.121183,-0.567667,0.719730,-0.596276,-0.196945,-1.069565,-0.037975,-0.532766,-0.993144,0.290782,-1.965020,1.006662,0.727382,-0.916638,0.294584,0.175633,-0.329569,-1.594364,-0.380805,-1.026010,0.348663,-1.128420,-0.848889,0.395881,-0.644095,1.032303,-0.416298,-0.381516,0.331897,-0.212423,0.405539,0.478982,0.986514,-0.606061,0.752937,-0.391577,0.972345,-0.699180,1.416896,-0.035409,0.200891,0.205325,-0.181144,1.049503,0.317013,0.544309,0.183808,1.482357,-0.893175,-0.682820,-0.231085,2.022681,-1.122003,1.718735,-0.309482,0.823394,-1.064353,-0.127237,0.102409,0.043388,1.402703,0.746605,0.802117,1.873780,-0.668953,-0.997651,0.942246,-1.835372,0.428325,-0.195086,2.256042,1.539681,-0.514522,0.279452,0.193586,-1.173854,0.624718,-2.064704,-0.910975,-1.272022,-0.672177,-0.952287,-1.032953,-1.134156,-0.157197,-0.596468,-2.594566,-1.247591,-0.051279,0.586029,-0.51076,-0.149608,-0.181149,-0.937995,-3.354257,0.488988,-0.470222,1.731971,-1.657018,-0.572079,-0.954330,-0.731785,0.635395,0.739349,-1.306867,2.841676,-0.067328,-2.420232,1.441512,-0.904469,1.303498,0.800713,-0.270663,-1.147452,0.850252,0.854831,-1.716410,-2.417039,0.530131,-0.489137,-0.554281,1.335108,-0.495867,1.762544,0.546254,-0.935553,0.930527,2.865101,0.626827,-1.588116,-0.142715,-1.279854,-1.412623,0.371282,1.185201,-1.728229,-2.364275,-0.595931,-1.741295,-0.907915,-2.007255,-4.515944,0.025911,-2.525274,-0.363382,2.548656,-1.997232,-1.749106,-1.763551,2.747723,2.370950,-0.002947,2.058105,0.688534,0.458410,0.169805,-0.862341,-4.480311,0.711578,1.076330,3.750731,-1.012084,4.708464,-1.142835,1.729280,-2.347064,-0.056713,-3.502054,5.159105,0.849958,1.107533,-0.747772,-1.268196,0.083161,0.034944,0.975879,-4.501689,-0.510125,4.242717,0.382817,-8.003626,-0.901595,-1.428074,-4.440095,-3.079094,-4.534797,4.100522,10.448793,-6.894423,1.286336,6.308175,-0.593790,1.110578,0.982747,2.188643,-3.282873,-4.767019,-3.611096,-2.470767,-1.611356,2.297540,0.725317,-1.443608,-0.946805,0.285272,-2.075893,-3.926974,-1.637788,-4.638062,2.779232,1.413041,-0.490583,7.522144,-4.165016,4.073620,-1.157172,3.519822,-0.596526,4.376914,4.479558,-0.357605,-1.925904,10.690581,3.108071,-0.784957,8.821456,-5.853478,-6.519961,-0.408366,2.792866,-7.319533,-1.655546,-20.423803,-6.685496,1.670839,-14.606997,-1.800523,24.132189,16.908558,9.110896,-1.945196,-4.387478,24.428373,17.336388,5.127117,-9.641279,22.741100,-11.064229,24.051641,10.331159,12.276607,9.227096,5.805639,-4.539077,8.659876,-1.060631,-0.643798,-6.404347,2.125967,6.644600,0.644470,-7.696475,3.069463,4.185759,0.743334,-1.387411,-4.195225,-1.306396,-5.950106,-13.238517,0.178233
1,1.184548,-1.581033,1.777569,-

In [10]:
article_matrix_df.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
7662811,-0.358318,-1.266898,-0.115116,0.763059,0.713006,-0.784260,-1.005553,-0.047356,0.685860,-0.797120,-0.806324,-0.221712,1.259992,-0.237436,1.179406,-0.726878,0.090642,1.225578,1.399777,0.121848,-0.388056,-1.337216,1.272953,1.702944,-0.62498,-0.080586,-0.204185,1.43663,0.516778,-1.000600,0.565639,0.812400,-0.455838,0.882743,0.252354,-0.684581,-0.538223,-0.578521,1.277407,0.417728,1.138446,-0.761334,0.909655,1.307129,0.536259,-0.326073,-0.780559,1.928360,-0.113701,0.951389,-1.043155,-0.726729,-0.258808,-1.414575,0.783656,-1.163518,0.870553,-0.935648,-0.799525,0.836843,0.518188,-1.056705,1.615284,1.103418,-0.501106,-1.075890,1.625494,1.822751,0.086480,-0.159644,-1.385113,-0.412606,-0.629446,1.135309,-0.061068,-0.244372,0.668623,-0.164709,-0.078412,-0.065244,1.287752,0.423566,-1.041172,-1.120917,-0.606599,-1.908998,-1.179654,2.003501,-1.518811,0.697839,-1.096643,-0.376338,-0.382601,0.688592,-0.819009,0.615035,-0.075623,-0.050948,-0.214962,1.623770,-0.864913,-1.207607,0.287706,-0.098911,-1.118886,-1.300501,1.011848,0.719185,1.254717,0.862834,0.434450,-0.410148,-1.491610,1.118517,-2.293207,0.627641,-2.259574,-0.455068,-3.056926,-2.549214,0.415097,-0.312829,2.917838,2.045774,0.148927,1.426258,1.036137,-2.338337,-2.918135,3.047282,-0.418137,1.42784,-0.915260,2.279778,-3.54468,-0.775453,-0.070702,0.567880,-1.252736,3.562377,1.862673,1.821876,-0.796917,-2.534385,0.459045,-1.223786,-2.463480,-1.326056,-1.650843,2.414696,-2.340174,0.082975,-0.656414,-0.445180,-0.611144,-0.505349,1.600636,2.873242,0.527345,1.077713,1.367972,-3.097965,2.463956,-1.202929,-4.112458,-0.503787,0.218564,-2.294107,-1.647082,1.458303,0.370740,0.055460,0.057220,-2.666892,-1.665880,0.341761,0.097000,-1.409934,-0.815934,-1.792042,1.962376,0.539422,-3.126810,4.269176,-3.523431,-1.549876,2.808787,-3.044948,1.809447,-2.743266,-1.816082,-2.705206,-0.216404,0.316312,-0.842260,-1.397785,-2.628934,3.029766,0.026877,-0.655035,-0.110204,1.178656,1.506050,-0.113088,-1.791010,-0.326535,0.882819,-0.587174,1.144226,-2.66242,-0.07615,2.908550,-2.287156,3.640100,-0.783909,0.873956,-2.803720,-0.922229,1.126898,3.681890,-2.574559,0.883177,-1.081153,1.511347,-0.943755,-1.147785,1.307851,1.650933,-2.281885,-1.673382,-0.726314,4.65008,-0.973241,-1.317519,0.084922,0.822928,-0.287604,2.742860,-0.531546,3.206332,1.393138,0.490890,-1.088799,-1.290722,0.220707,0.647000,-0.016062,-0.718720,-0.973011,-0.409868,0.117330,-1.700233,-0.786038,0.028505,0.887542,0.167491,1.674013,1.756230,-1.187147,-0.549629,-0.744133,0.487791,-0.463145,-0.686940,-0.858300,-0.588848,0.599170,-0.369356,-1.441070,0.401900,0.410071,0.697208,0.102579,-0.363132,0.076454,-0.193966,0.022932,0.538259,-0.220087,0.452328,-0.557674,0.506213,1.154371,0.011966,-0.574812,0.145060,0.439228,-1.068391,-0.743637,-0.241305,0.979040,-1.188981,-1.424874,-9.136146,-0.694681,-1.344703,-0.664818,-1.527619,-3.073687,0.025204
7787495,-0.392456,0.302272,0.053423,0.039

In [12]:
# Save the cosine similarity matrix to pickle file cause is very big

user_matrix_df_file_path = f'./files/pickle/user_matrix_{type_}_{size}_{fillna_value}.pkl'
article_matrix_df_file_path = f'./files/pickle/article_matrix_{type_}_{size}_{fillna_value}.pkl'

user_matrix_df.to_pickle(user_matrix_df_file_path)
article_matrix_df.to_pickle(article_matrix_df_file_path)

## Other

In [5]:
# # Define a function to convert strings back to lists
# def str_to_list(s):
#     try:
#         temp_lst = s.replace('[', '').replace(']','').replace(' ', '').split(',')
        
#         return [float(x) for x in temp_lst]
#     except:
#         return s


# for col in tqdm(interaction_matrix.columns.to_list()):
#     interaction_matrix[col] = interaction_matrix[col].apply(str_to_list)

/var/folders/ld/c07vs56s18bc0h2krlvl0myr0000gn/T/ipykernel_2337/1990072283.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for col in tqdm(interaction_matrix.columns.to_list()):


  0%|          | 0/4248 [00:00<?, ?it/s]